# HW: X-ray images classification
--------------------------------------

Before you begin, open Mobaxterm and connect to triton with the user and password you were give with. Activate the environment `2ndPaper` and then type the command `pip install scikit-image`.

In this assignment you will be dealing with classification of 32X32 X-ray images of the chest. The image can be classified into one of four options: lungs (l), clavicles (c), and heart (h) and background (b). Even though those labels are dependent, we will treat this task as multiclass and not as multilabel. The dataset for this assignment is located on a shared folder on triton (`/MLdata/MLcourse/X_ray/'`).

In [1]:
import os
import numpy as np
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Dropout
from tensorflow.keras.layers import Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *

from tensorflow.keras.initializers import Constant
from tensorflow.keras.datasets import fashion_mnist
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from skimage.io import imread

from skimage.transform import rescale, resize, downscale_local_mean
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import tensorflow as tf

config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
def preprocess(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        src = imread(os.path.join(datapath, fn),1)
        img = resize(src,(32,32),order = 3)
        
        images[ii,:,:,0] = img
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    BaseImages = images
    BaseY = Y
    return BaseImages, BaseY

In [4]:
def preprocess_train_and_val(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        images[ii,:,:,0] = imread(os.path.join(datapath, fn),1)
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    return images, Y

In [5]:
#Loading the data for training and validation:
src_data = 'MLdata/MLcourse/X_ray/'
train_path = src_data + 'train'
val_path = src_data + 'validation'
test_path = src_data + 'test'
BaseX_train , BaseY_train = preprocess_train_and_val(train_path)
BaseX_val , BaseY_val = preprocess_train_and_val(val_path)
X_test, Y_test = preprocess(test_path)

In [6]:
keras.backend.clear_session()

### PART 1: Fully connected layers 
--------------------------------------

---
<span style="color:red">***Task 1:***</span> *NN with fully connected layers. 

Elaborate a NN with 2 hidden fully connected layers with 300, 150 neurons and 4 neurons for classification. Use ReLU activation functions for the hidden layers and He_normal for initialization. Don't forget to flatten your image before feedforward to the first dense layer. Name the model `model_relu`.*

---

In [7]:
#--------------------------Impelment your code here:-------------------------------------
initializer = tf.keras.initializers.HeNormal()
input_shape = BaseX_train[0].shape
model_relu = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(300, activation='relu', kernel_initializer=initializer),
    tf.keras.layers.Dense(150, activation='relu', kernel_initializer=initializer),
    tf.keras.layers.Dense(4)
])
#----------------------------------------------------------------------------------------

In [8]:
model_relu.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 300)               307500    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 604       
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

Compile the model with the optimizer above, accuracy metric and adequate loss for multiclass task. Train your model on the training set and evaluate the model on the testing set. Print the accuracy and loss over the testing set.

In [10]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt,
              loss='categorical_crossentropy', metrics=['accuracy'])

model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

results = model_relu.evaluate(X_test,Y_test)
print('test loss, test acc:', results)
#----------------------------------------------------------------------------------------

Epoch 1/25
102/102 [==============================] - 0s 2ms/step - loss: 2.7868 - accuracy: 0.4126
Epoch 2/25
102/102 [==============================] - 0s 1ms/step - loss: 5.2851 - accuracy: 0.5063
Epoch 3/25
102/102 [==============================] - 0s 1ms/step - loss: 5.4206 - accuracy: 0.4972
Epoch 4/25
102/102 [==============================] - 0s 1ms/step - loss: 4.5447 - accuracy: 0.4015
Epoch 5/25
102/102 [==============================] - 0s 1ms/step - loss: 3.7627 - accuracy: 0.4430
Epoch 6/25
102/102 [==============================] - 0s 1ms/step - loss: 3.4237 - accuracy: 0.4455
Epoch 7/25
102/102 [==============================] - 0s 1ms/step - loss: 3.5954 - accuracy: 0.4442
Epoch 8/25
102/102 [==============================] - 0s 1ms/step - loss: 3.0398 - accuracy: 0.3133
Epoch 9/25
102/102 [==============================] - 0s 1ms/step - loss: 2.9445 - accuracy: 0.3193
Epoch 10/25
102/102 [==============================] - 0s 1ms/step - loss: 3.4172 - accuracy: 0.4019

---
<span style="color:red">***Task 2:***</span> *Activation functions.* 

Change the activation functions to LeakyRelu or tanh or sigmoid. Name the new model `new_a_model`. Explain how it can affect the model.*

---

In [11]:
#--------------------------Impelment your code here:-------------------------------------
initializer = tf.keras.initializers.HeNormal()
input_shape = BaseX_train[0].shape
new_a_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(300, activation='sigmoid', kernel_initializer=initializer),
    tf.keras.layers.Dense(150, activation='sigmoid', kernel_initializer=initializer),
    tf.keras.layers.Dense(4)
])
#----------------------------------------------------------------------------------------

### Answer

Rectified Linear Unit (ReLU) convert linear outputs of a neuron into nonlinear outputs by outputting x for all x >= 0 and 0 for all x < 0. 
In other words, it equals max(x, 0). This simplicity makes it more attractive than the Sigmoid activation function and the Tangens hyperbolicus (Tanh) activation function, which use more difficult formulas and are computationally more expensive. 
In addition, ReLU is not sensitive to vanishing gradients, whereas the other two are, slowing down learning in your network and also known to generalize well.
This does however not mean that ReLU itself does not have certain challenges:
- It tends to produce very large values given its non-boundedness on the upside of the domain. Theoretically, infinite inputs produce infinite outputs.
- If a neuron’s weights are moved towards the zero output, it may be the case that they eventually will no longer be capable of recovering from this. They will then continually output zeros. This is especially the case when our network is poorly initialized, or when your data is poorly normalized.
- Small values, even the non-positive ones, may be of value; they can help capture patterns underlying the dataset. With ReLU, this cannot be done, since all outputs smaller than zero are zero.

We prefer the Sigmoid function because it outputs between (0,1). When estimating a probability, this is perfect, because probabilities have a very similar range of [0,1]. Especially in binary classification problems, when we effectively estimate the probability that the output is of some class, Sigmoid functions allow us to give a very weighted estimate.




In [12]:
new_a_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               307500    
_________________________________________________________________
dense_4 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 604       
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


---
<span style="color:red">***Task 3:***</span> *Number of epochs.* 

Train the new model using 25 and 40 epochs. What difference does it makes in term of performance? Remember to save the compiled model for having initialized weights for every run as we did in tutorial 12. Evaluate each trained model on the test set*

---

In [13]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

In [15]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,
              loss='categorical_crossentropy', metrics=['accuracy'])

new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

results = new_a_model.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "25_epochs_final_weights.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
print('Saved trained model at %s ' % model_path)
#-----------------------------------------------------------------------------------------

Epoch 1/25
102/102 [==============================] - 0s 2ms/step - loss: 4.7579 - accuracy: 0.2502
Epoch 2/25
102/102 [==============================] - 0s 1ms/step - loss: 5.3469 - accuracy: 0.2502
Epoch 3/25
102/102 [==============================] - 0s 1ms/step - loss: 9.1151 - accuracy: 0.2502
Epoch 4/25
102/102 [==============================] - 0s 1ms/step - loss: 9.6029 - accuracy: 0.2502
Epoch 5/25
102/102 [==============================] - 0s 1ms/step - loss: 9.9712 - accuracy: 0.2502
Epoch 6/25
102/102 [==============================] - 0s 1ms/step - loss: 10.2923 - accuracy: 0.2502
Epoch 7/25
102/102 [==============================] - 0s 1ms/step - loss: 10.4392 - accuracy: 0.2502
Epoch 8/25
102/102 [==============================] - 0s 1ms/step - loss: 10.4989 - accuracy: 0.2502
Epoch 9/25
102/102 [==============================] - 0s 1ms/step - loss: 10.5786 - accuracy: 0.2502
Epoch 10/25
102/102 [==============================] - 0s 1ms/step - loss: 10.6309 - accuracy: 0

In [16]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 40

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

In [18]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,
              loss='categorical_crossentropy', metrics=['accuracy'])

new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

results = new_a_model.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "40_epochs_final_weights.h5"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
print('Saved trained model at %s ' % model_path)
#-----------------------------------------------------------------------------------------

Epoch 1/40
102/102 [==============================] - 0s 2ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 2/40
102/102 [==============================] - 0s 2ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 3/40
102/102 [==============================] - 0s 1ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 4/40
102/102 [==============================] - 0s 1ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 5/40
102/102 [==============================] - 0s 2ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 6/40
102/102 [==============================] - 0s 1ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 7/40
102/102 [==============================] - 0s 1ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 8/40
102/102 [==============================] - 0s 1ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 9/40
102/102 [==============================] - 0s 1ms/step - loss: 11.0541 - accuracy: 0.2502
Epoch 10/40
102/102 [==============================] - 0s 1ms/step - loss: 11.0541 - accura

---
<span style="color:red">***Task 4:***</span> *Mini-batches.* 

Build the `model_relu` again and run it with a batch size of 32 instead of 64. What are the advantages of the mini-batch vs. SGD?*

---

### Answer

(mini-) Batch Gradient Descent involves calculations over the full training set at each step as a result of which it is very slow on very large training data. Thus, it becomes very computationally expensive to do Batch GD. However, this is great for convex or relatively smooth error manifolds. Also, Batch GD scales well with the number of features.

SGD tries to solve the main problem in Batch Gradient descent which is the usage of whole training data to calculate gradients as each step. SGD is stochastic in nature, i.e, it picks up a “random” instance of training data at each step and then computes the gradient making it much faster as there is much fewer data to manipulate at a single time, unlike Batch GD.
There is a downside of the Stochastic nature of SGD: once it reaches close to the minimum value then it doesn’t settle down, instead bounces around which gives us a good value for model parameters but not optimal.

In [19]:
keras.backend.clear_session()

In [20]:
#--------------------------Impelment your code here:-------------------------------------
initializer = tf.keras.initializers.HeNormal()
input_shape = BaseX_train[0].shape
model_relu = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(300, activation='relu', kernel_initializer=initializer),
    tf.keras.layers.Dense(150, activation='relu', kernel_initializer=initializer),
    tf.keras.layers.Dense(4)
])
#----------------------------------------------------------------------------------------

In [21]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

In [23]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt,
              loss='categorical_crossentropy', metrics=['accuracy'])

model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

results = model_relu.evaluate(X_test,Y_test)
print('test loss, test acc:', results)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 [==============================] - 0s 1ms/step - loss: 6.6209 - accuracy: 0.2660
Epoch 2/50
203/203 [==============================] - 0s 1ms/step - loss: 8.2241 - accuracy: 0.2777
Epoch 3/50
203/203 [==============================] - 0s 1ms/step - loss: 8.7193 - accuracy: 0.2575
Epoch 4/50
203/203 [==============================] - 0s 1ms/step - loss: 8.4202 - accuracy: 0.2502
Epoch 5/50
203/203 [==============================] - 0s 1ms/step - loss: 9.5240 - accuracy: 0.2550
Epoch 6/50
203/203 [==============================] - 0s 1ms/step - loss: 6.0272 - accuracy: 0.3202
Epoch 7/50
203/203 [==============================] - 0s 1ms/step - loss: 8.9772 - accuracy: 0.2967
Epoch 8/50
203/203 [==============================] - 0s 1ms/step - loss: 8.8839 - accuracy: 0.2621
Epoch 9/50
203/203 [==============================] - 0s 1ms/step - loss: 6.7142 - accuracy: 0.2502
Epoch 10/50
203/203 [==============================] - 0s 1ms/step - loss: 7.2666 - accuracy: 0.2683

---
<span style="color:red">***Task 5:***</span> *Batch normalization.* 

Build the `new_a_model` again and add batch normalization layers. How does it impact your results?*

---

In [24]:
keras.backend.clear_session()

In [25]:
#--------------------------Impelment your code here:-------------------------------------
initializer = tf.keras.initializers.HeNormal()
input_shape = BaseX_train[0].shape
new_a_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(300, activation='sigmoid', kernel_initializer=initializer),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.Dense(150, activation='sigmoid', kernel_initializer=initializer),
    tf.keras.layers.BatchNormalization(axis=1),
    tf.keras.layers.Dense(4)
])
#---------------------------------------------------------------------------------------

In [26]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)
#Compile the network: 
new_a_model.compile(optimizer=AdamOpt,
              loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
#Preforming the training by using fit 
#--------------------------Impelment your code here:-------------------------------------
new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs)

results = new_a_model.evaluate(X_test,Y_test)
print('test loss, test acc:', results)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 [==============================] - 0s 2ms/step - loss: 4.5452 - accuracy: 0.2394
Epoch 2/50
203/203 [==============================] - 0s 2ms/step - loss: 3.1007 - accuracy: 0.2618
Epoch 3/50
203/203 [==============================] - 0s 2ms/step - loss: 2.9592 - accuracy: 0.2607
Epoch 4/50
203/203 [==============================] - 0s 1ms/step - loss: 2.5001 - accuracy: 0.2359
Epoch 5/50
203/203 [==============================] - 0s 2ms/step - loss: 2.5152 - accuracy: 0.2388
Epoch 6/50
203/203 [==============================] - 0s 2ms/step - loss: 2.4063 - accuracy: 0.2505
Epoch 7/50
203/203 [==============================] - 0s 1ms/step - loss: 2.5225 - accuracy: 0.2488
Epoch 8/50
203/203 [==============================] - 0s 1ms/step - loss: 2.5246 - accuracy: 0.2739
Epoch 9/50
203/203 [==============================] - 0s 1ms/step - loss: 2.4773 - accuracy: 0.2878
Epoch 10/50
203/203 [==============================] - 0s 2ms/step - loss: 2.4154 - accuracy: 0.2884

### PART 2: Convolutional Neural Network (CNN)
------------------------------------------------------------------------------------

---
<span style="color:red">***Task 1:***</span> *2D CNN.* 

Have a look at the model below and answer the following:
* How many layers does it have?
* How many filter in each layer?
* Would the number of parmaters be similar to a fully connected NN?
* Is this specific NN performing regularization?

---

In [28]:
def get_net(input_shape,drop,dropRate,reg):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

In [29]:
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2

In [30]:
NNet=get_net(input_shape,drop,dropRate,reg)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)      

### Answer
1. How many layers does it have?

    The following network has 24 layers

2. How many filters in each layer?

    Conv2D_1 has 64 filters, Conv2D_2 and Conv2D_3 have 128 filters, Conv2D_4 and Conv2D_5 have 256 filters

3. Would the number of parmaters be similar to a fully connected NN?

    Num of parmaters in a Conv layer = ((shape of width of the filter * shape of height of the filter * number of filters in the previous layer+1)*number of filters)
    
    Num of parmaters in a Fully connected = ((current layer neurons * previous layer neurons)+1*c)

    As we can cealry see, the number of parmaters of a fully connected NN would increas dramatically.

4. Is this specific NN performing regularization?

    This specific NN performs L2 regularization with value of 1e-2

In [31]:
from tensorflow.keras.optimizers import *
import os
from tensorflow.keras.callbacks import *

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

In [32]:
#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
#NNet.save(model_fn)

Epoch 1/25
102/102 [==============================] - 37s 362ms/step - loss: 7.9929 - acc: 0.4476 - val_loss: 7.8973 - val_acc: 0.2500
Epoch 2/25
102/102 [==============================] - 37s 362ms/step - loss: 7.6147 - acc: 0.5436 - val_loss: 8.0183 - val_acc: 0.2500
Epoch 3/25
102/102 [==============================] - 37s 362ms/step - loss: 7.4466 - acc: 0.5763 - val_loss: 8.1156 - val_acc: 0.2697
Epoch 4/25
102/102 [==============================] - 37s 361ms/step - loss: 7.3275 - acc: 0.6166 - val_loss: 8.1497 - val_acc: 0.2951
Epoch 5/25
102/102 [==============================] - 37s 360ms/step - loss: 7.2402 - acc: 0.6361 - val_loss: 8.0580 - val_acc: 0.2766
Epoch 6/25
102/102 [==============================] - 37s 361ms/step - loss: 7.1534 - acc: 0.6673 - val_loss: 7.8777 - val_acc: 0.3247
Epoch 7/25
102/102 [==============================] - 38s 370ms/step - loss: 7.1051 - acc: 0.6784 - val_loss: 7.7426 - val_acc: 0.3530
Epoch 8/25
102/102 [==============================] - 3

In [ ]:
# NNet.load_weights('Weights_1.h5')

In [33]:
results = NNet.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

6/6 [==============================] - 0s 28ms/step - loss: 7.9439 - acc: 0.3029
test loss, test acc: [7.943883419036865, 0.3028571307659149]


---
<span style="color:red">***Task 2:***</span> *Number of filters* 

Rebuild the function `get_net` to have as an input argument a list of number of filters in each layers, i.e. for the CNN defined above the input should have been `[64, 128, 128, 256, 256]`. Now train the model with the number of filters reduced by half. What were the results.

---

In [34]:
#--------------------------Impelment your code here:-------------------------------------
def get_net(input_shape,drop,dropRate,reg, filters):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=filters[0], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filters[1], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filters[2], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=filters[3], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=filters[4], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model


input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
filters = np.array([64, 128, 128, 256, 256])
filters = filters/2

NNet=get_net(input_shape,drop,dropRate,reg,filters)

from tensorflow.keras.optimizers import *
import os
from tensorflow.keras.callbacks import *

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)

results = NNet.evaluate(X_test,Y_test)
print('test loss, test acc:', results)
#----------------------------------------------------------------------------------------

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 64)       

That's all folks! See you :)